In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# Espero que este notebook funcione para poupar um pouco da minha sanidade mental que me resta.
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from datetime import datetime # convert to datetime format

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/graph-rename/excelteste.xlsx
/kaggle/input/grafo1/Grafo1.xlsx
/kaggle/input/datasetex/GrafoEx.xlsx


Na linha abaixo deve ser inserido o caminho junto ao nome do arquivo.

Exemplo:

**patch = "/kaggle/input/grafo1/Grafo1.xlsx"**

In [2]:
patch = "/kaggle/input/datasetex/GrafoEx.xlsx"

O código abaixo lê o arquivo e mostra o inicio da tabela, apenas para averiguar se é o arquivo correto.

In [3]:
tabela = pd.read_excel(patch)
tabela.head()

,Tempo,Tempo_Fim,Emissor,Receptor
0,00:02:40,00:04:41,Gura,Grupo
1,00:04:41,00:05:27,Amelia,Grupo
2,00:05:27,00:05:29,Gura,Amelia
3,00:05:29,00:05:34,Gura,Watson
4,00:05:34,0,Watson,Gura


Nesta parte, está sendo realizado um processo de normalização das colunas "Emissor" e "Receptor". Inicialmente é removido os acentos, logo após é ajustado para o padrão de minusculos e maiusculos.

**Nota: caracteres diferentes do alfabeto (ex: ? !) serão substituidos por ?** 

In [4]:
tabela['Emissor'] = tabela['Emissor'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
tabela['Receptor'] = tabela['Receptor'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

tabela['Emissor'] = tabela['Emissor'].str.title()
tabela['Receptor'] = tabela['Receptor'].str.title()

tabela['Emissor'] = tabela['Emissor'].replace([''],['?'])
tabela['Receptor'] = tabela['Receptor'].replace([''],['?'])

tabela.head()

,Tempo,Tempo_Fim,Emissor,Receptor
0,00:02:40,00:04:41,Gura,Grupo
1,00:04:41,00:05:27,Amelia,Grupo
2,00:05:27,00:05:29,Gura,Amelia
3,00:05:29,00:05:34,Gura,Watson
4,00:05:34,0,Watson,Gura


Esta parte está mudando o tipo da entrada para *datetime64*, com a finalidade de realizar a subtração futuramente.

In [5]:
start_time = pd.to_datetime(tabela['Tempo'], format='%H:%M:%S')
start_time.head()

0   1900-01-01 00:02:40
1   1900-01-01 00:04:41
2   1900-01-01 00:05:27
3   1900-01-01 00:05:29
4   1900-01-01 00:05:34
Name: Tempo, dtype: datetime64[ns]

Na tabela do word/excel, é recomendado escrever 0 em Tempo_Fim, apenas para agilizar o tempo de escrita, porém para converter para o formato de *datetime*, é necessário utilizar o formato padrão '00:00:00', logo a linha abaixo substitui os '0' por '00:00:00'. 

In [6]:
tabela.Tempo_Fim = tabela['Tempo_Fim'].replace([0],['0:00:00'])

tabela.head()

,Tempo,Tempo_Fim,Emissor,Receptor
0,00:02:40,00:04:41,Gura,Grupo
1,00:04:41,00:05:27,Amelia,Grupo
2,00:05:27,00:05:29,Gura,Amelia
3,00:05:29,00:05:34,Gura,Watson
4,00:05:34,0:00:00,Watson,Gura


Por fim, assim como já feito anteriormente, é convertido para o formato *datetime*

In [7]:
final_time = pd.to_datetime(tabela['Tempo_Fim'], format='%H:%M:%S')

tabela.head()


,Tempo,Tempo_Fim,Emissor,Receptor
0,00:02:40,00:04:41,Gura,Grupo
1,00:04:41,00:05:27,Amelia,Grupo
2,00:05:27,00:05:29,Gura,Amelia
3,00:05:29,00:05:34,Gura,Watson
4,00:05:34,0:00:00,Watson,Gura


Abaixo é criado uma nova tabela, apenas para fins organizacionais com o Tempo e o Tempo_Fim.

In [8]:
time_tabel = pd.concat([start_time, final_time], axis = 1)

time_tabel.head()

,Tempo,Tempo_Fim
0,1900-01-01 00:02:40,1900-01-01 00:04:41
1,1900-01-01 00:04:41,1900-01-01 00:05:27
2,1900-01-01 00:05:27,1900-01-01 00:05:29
3,1900-01-01 00:05:29,1900-01-01 00:05:34
4,1900-01-01 00:05:34,1900-01-01 00:00:00


A linha abaixo, adiciona uma nova coluna na tabela time_tabel com a diferença entre o Tempo_Fim e o Tempo.

**Nota: Repare que haverá números negativos, que se dão pelo fato de serem exatamente os tempos que duram 1 segundo, ou seja, na tabela do word/excel eles estão representados como por exemplo Tempo: 00:02:40 e Tempo_Fim: 00:00:00, logo fazendo a subtração entre o Tempo_Fim e o Tempo ocasionária este fator negativo que será convertido mais a frente.**

In [9]:
time_tabel['Total_Time'] = time_tabel.Tempo_Fim - time_tabel.Tempo

time_tabel.head(10)

,Tempo,Tempo_Fim,Total_Time
0,1900-01-01 00:02:40,1900-01-01 00:04:41,0 days 00:02:01
1,1900-01-01 00:04:41,1900-01-01 00:05:27,0 days 00:00:46
2,1900-01-01 00:05:27,1900-01-01 00:05:29,0 days 00:00:02
3,1900-01-01 00:05:29,1900-01-01 00:05:34,0 days 00:00:05
4,1900-01-01 00:05:34,1900-01-01 00:00:00,-1 days +23:54:26
5,1900-01-01 00:05:35,1900-01-01 00:05:41,0 days 00:00:06
6,1900-01-01 00:05:43,1900-01-01 00:00:00,-1 days +23:54:17
7,1900-01-01 00:05:43,1900-01-01 00:05:49,0 days 00:00:06
8,1900-01-01 00:05:49,1900-01-01 00:00:00,-1 days +23:54:11
9,1900-01-01 00:05:55,1900-01-01 00:05:57,0 days 00:00:02


É adicionado uma nova coluna a tabela time_tabel, que é a converção do Total_Time para um total em segundos.

In [10]:
time_tabel['Total_Seconds'] = time_tabel['Total_Time'].dt.total_seconds()

time_tabel.head(10)

,Tempo,Tempo_Fim,Total_Time,Total_Seconds
0,1900-01-01 00:02:40,1900-01-01 00:04:41,0 days 00:02:01,121.0
1,1900-01-01 00:04:41,1900-01-01 00:05:27,0 days 00:00:46,46.0
2,1900-01-01 00:05:27,1900-01-01 00:05:29,0 days 00:00:02,2.0
3,1900-01-01 00:05:29,1900-01-01 00:05:34,0 days 00:00:05,5.0
4,1900-01-01 00:05:34,1900-01-01 00:00:00,-1 days +23:54:26,-334.0
5,1900-01-01 00:05:35,1900-01-01 00:05:41,0 days 00:00:06,6.0
6,1900-01-01 00:05:43,1900-01-01 00:00:00,-1 days +23:54:17,-343.0
7,1900-01-01 00:05:43,1900-01-01 00:05:49,0 days 00:00:06,6.0
8,1900-01-01 00:05:49,1900-01-01 00:00:00,-1 days +23:54:11,-349.0
9,1900-01-01 00:05:55,1900-01-01 00:05:57,0 days 00:00:02,2.0


A linha abaixo faz o reparo dos números negativos e adiciona uma unidade na soma. Todo número negativo é convertido em 1 pelo o que já foi explicado anteriormente, a correção adicionando +1 se da pelo motivo que:

Ao fazermos a subtração de Tempo: 2:40 e Tempo_Fim: 2:41 teremos 1 segundo de diferença, porém olhando no contexto de análise de vídeo, não estamos utilizando a diferença de tempo, mas sim a quantidade de segundos que tiveram no total, ou seja, tivemos o segundo 2:40 e o segundo 2:41, um total de 2 segundos, logo é necessário fazer a correção somando 1 nos casos em que Total_Seconds é maior que zero

In [11]:
time_tabel['Total_Reajuste'] = time_tabel['Total_Seconds'].apply(lambda x: 1 if x <= 0 else x + 1)
time_tabel.head(10)

,Tempo,Tempo_Fim,Total_Time,Total_Seconds,Total_Reajuste
0,1900-01-01 00:02:40,1900-01-01 00:04:41,0 days 00:02:01,121.0,122.0
1,1900-01-01 00:04:41,1900-01-01 00:05:27,0 days 00:00:46,46.0,47.0
2,1900-01-01 00:05:27,1900-01-01 00:05:29,0 days 00:00:02,2.0,3.0
3,1900-01-01 00:05:29,1900-01-01 00:05:34,0 days 00:00:05,5.0,6.0
4,1900-01-01 00:05:34,1900-01-01 00:00:00,-1 days +23:54:26,-334.0,1.0
5,1900-01-01 00:05:35,1900-01-01 00:05:41,0 days 00:00:06,6.0,7.0
6,1900-01-01 00:05:43,1900-01-01 00:00:00,-1 days +23:54:17,-343.0,1.0
7,1900-01-01 00:05:43,1900-01-01 00:05:49,0 days 00:00:06,6.0,7.0
8,1900-01-01 00:05:49,1900-01-01 00:00:00,-1 days +23:54:11,-349.0,1.0
9,1900-01-01 00:05:55,1900-01-01 00:05:57,0 days 00:00:02,2.0,3.0


Abaixo é feita uma nova tabela, juntando o Total_Reajuste, Emissor e Receptor.

**Nota: TODA E QUALQUER linha que tenha INFORMAÇÃO FALTANDO será removida após executar a próxima linha**, isso se deve pelo fato de ao criar a tabela no word, geralmente são puladas linhas onde está indescritivel quem são os emissores e receptores, tendo apenas o tempo como informação ou alguma anotação.

In [12]:
Converter_Grafo = pd.concat([time_tabel['Total_Reajuste'], tabela['Emissor'], tabela['Receptor']], axis = 1)

Converter_Grafo = Converter_Grafo.dropna()

Abaixo, é gerado uma nova tabela, porém já repetindo a quantidade de interações que foram obtidas para gerar o grafo.

In [13]:
Converter_Grafo2 = Converter_Grafo.loc[Converter_Grafo.index.repeat(Converter_Grafo.Total_Reajuste)].reset_index(drop=True)
Converter_Grafo2.head()

,Total_Reajuste,Emissor,Receptor
0,122.0,Gura,Grupo
1,122.0,Gura,Grupo
2,122.0,Gura,Grupo
3,122.0,Gura,Grupo
4,122.0,Gura,Grupo


Por fim, está a tabela final pronta para ser utilizada nos programas que geram as redes.

In [14]:
Convertido = pd.concat([Converter_Grafo2['Emissor'], Converter_Grafo2['Receptor']], axis = 1)
Convertido.head()

,Emissor,Receptor
0,Gura,Grupo
1,Gura,Grupo
2,Gura,Grupo
3,Gura,Grupo
4,Gura,Grupo


A linha abaixo baixa/cria o arquivo desejado.

In [15]:
Convertido.to_excel("Grafo.xlsx")

O código a seguir apenas remove **TODAS** as linhas que contenham "Grupo".

In [18]:
Convertido_SemGrupo = Convertido[(Convertido["Receptor"].str.contains("Grupo") == False)].reset_index(drop=True)
Convertido_SemGrupo = Convertido_SemGrupo[(Convertido_SemGrupo["Emissor"].str.contains("Emissor") == False)].reset_index(drop=True)

Convertido_SemGrupo.head()

,Emissor,Receptor
0,Gura,Amelia
1,Gura,Amelia
2,Gura,Amelia
3,Gura,Watson
4,Gura,Watson


A linha abaixo baixa/cria o arquivo desejado.

In [17]:
Convertido_SemGrupo.to_excel("Grafo_SemGrupo.xlsx")